In [0]:
# from theano.sandbox import cuda
# Skip GPU check
# esnure GPU is on on colab...

In [119]:
! pip install bcolz

In [0]:
%matplotlib inline
#import utils; reload(utils)
#from utils import *
# check if you need utils down there...
from __future__ import division, print_function

In [0]:
# DO it the hard way.. 
# Import things as you need.. so that you know what you are importing..
# rahter than importing the whole utils.py file.. an dbeing clueless about where this fn came from 
import numpy as np
import os, re

In [0]:
model_path = 'data/imdb/models/'
%mkdir -p $model_path

## Setup data

Going to look at the IMDB dataset, which contains movie reviews from IMDB, along with their sentiment.  
Keras comes with some helper functions for this dataset.

In [0]:
from keras.datasets import imdb
idx = imdb.get_word_index()

This is the word list:

In [124]:
type(idx)

dict

In [125]:
idx_arr = sorted(idx, key=idx.get)
idx_arr[:10]
#u is for unicode

[u'the', u'and', u'a', u'of', u'to', u'is', u'br', u'in', u'it', u'i']

...and this is the mapping from id to word

In [0]:
idx2word = {v: k for k, v in idx.iteritems()}

In [127]:
type(idx2word)

dict

In [128]:
first2pairs = {k: idx2word[k] for k in idx2word.keys()[:2]}
first2pairs

{1: u'the', 2: u'and'}

We download the reviews using code copied from keras.datasets:  
There no neater nack to do this..

In [0]:
# part of utils
from keras.utils.data_utils import get_file
import pickle
#end of part of utils

path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263') # cool.. very with an md5 hash :D
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

In [130]:
type(x_train)
isinstance(x_train, list)
isinstance(x_train[0][0], int)

True

In [154]:
x_train[0][0:3]

[23022, 309, 6]

In [132]:
len(x_train)

25000

Here's the 1st review. As you see, the words have been replaced by ids. The ids can be looked up in idx2word.

In [133]:
', '.join(map(str, x_train[0]))

'23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215'

The first word of the first review is 23022. Let's see what that is.

In [134]:
idx2word[23022] # idx2word is a dict that has the mappings

u'bromwell'

Here's the whole review, mapped from ids to words.

In [135]:
' '.join([idx2word[o] for o in x_train[0]])

u"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

The labels are 1 for positive, 0 for negative.

In [136]:
labels_train[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Reduce vocab size by setting rare words to max index.

In [0]:
vocab_size = 5000

trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_train]
test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_test]

In [138]:
trn[0] # so the rarer words are all set to 4999

array([4999,  309,    6,    3, 1069,  209,    9, 2175,   30,    1,  169,
         55,   14,   46,   82, 4999,   41,  393,  110,  138,   14, 4999,
         58, 4477,  150,    8,    1, 4999, 4999,  482,   69,    5,  261,
         12, 4999, 4999, 2003,    6,   73, 2436,    5,  632,   71,    6,
       4999,    1, 4999,    5, 2004, 4999,    1, 4999, 1534,   34,   67,
         64,  205,  140,   65, 1232, 4999, 4999,    1, 4999,    4,    1,
        223,  901,   29, 3024,   69,    4,    1, 4999,   10,  694,    2,
         65, 1534,   51,   10,  216,    1,  387,    8,   60,    3, 1472,
       3724,  802,    5, 3521,  177,    1,  393,   10, 1238, 4999,   30,
        309,    3,  353,  344, 2989,  143,  130,    5, 4999,   28,    4,
        126, 4999, 1472, 2375,    5, 4999,  309,   10,  532,   12,  108,
       1470,    4,   58,  556,  101,   12, 4999,  309,    6,  227, 4187,
         48,    3, 2237,   12,    9,  215])

Look at distribution of lengths of sentences.

In [139]:
lens = np.array(map(len, trn))
(lens.max(), lens.min(), lens.mean())

(2493, 10, 237.71364)

Pad (with zero) or truncate each sentence to make consistent length.

In [0]:
from keras.preprocessing import  sequence

seq_len = 500

trn = sequence.pad_sequences(trn, maxlen=seq_len, value=0)
test = sequence.pad_sequences(test, maxlen=seq_len, value=0)

This results in nice rectangular matrices that can be passed to ML algorithms.   
Reviews shorter than 500 words are pre-padded with zeros, those greater are truncated.

In [141]:
trn.shape

(25000, 500)

## Create simple models

### Single hidden layer NN

The simplest model that tends to give reasonable results is a single hidden layer net. [the dense.. fully connected layer ] 
*Note*:: that we can't expect to get any useful results by feeding word ids directly into a neural net   
so instead we use an embedding to replace them with a vector of 32 (initially random) floats for each word in the vocab.  
embedding..:: the vector of weights associated with an enitity

In [0]:
from keras.models import Sequential
from keras.layers import Embedding, Input
from keras.layers.core import Flatten, Dense, Dropout
from keras.optimizers import Adam

from keras.layers.convolutional import *

In [0]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])# op: positive or negative

In [144]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
flatten_5 (Flatten)          (None, 16000)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 100)               1600100   
_________________________________________________________________
dropout_10 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 101       
Total params: 1,760,201
Trainable params: 1,760,201
Non-trainable params: 0
_________________________________________________________________


In [145]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 7s 278us/step - loss: 0.4640 - acc: 0.7540 - val_loss: 0.3022 - val_acc: 0.8718
Epoch 2/2
25000/25000 [==============================] - 7s 263us/step - loss: 0.1887 - acc: 0.9298 - val_loss: 0.3188 - val_acc: 0.8653


The [stanford paper](http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf) that this dataset is from cites a state of the art accuracy (without unlabelled data) of 0.883. So we're short of that, but on the right track.

### Single conv layer with max pooling

A CNN is likely to work better, since it's designed to take advantage of ordered data.   
We'll need to use a 1D CNN, since a sequence of words is 1D.

In [146]:
conv1 = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len, dropout=0.2),
    Dropout(0.2),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.2),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 5, padding="same", activation="relu")`
  after removing the cwd from sys.path.


In [0]:
conv1.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [148]:
conv1.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=4, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 9s 373us/step - loss: 0.5049 - acc: 0.7239 - val_loss: 0.2816 - val_acc: 0.8798
Epoch 2/4
25000/25000 [==============================] - 9s 357us/step - loss: 0.2647 - acc: 0.9030 - val_loss: 0.2717 - val_acc: 0.8850
Epoch 3/4
20224/25000 [=======================>......] - ETA: 1s - loss: 0.2055 - acc: 0.9236

25000/25000 [==============================] - 9s 355us/step - loss: 0.2080 - acc: 0.9223 - val_loss: 0.2627 - val_acc: 0.8897
Epoch 4/4
25000/25000 [==============================] - 9s 355us/step - loss: 0.1779 - acc: 0.9364 - val_loss: 0.2777 - val_acc: 0.8918


That's well past the Stanford paper's accuracy - another win for CNNs!
amlost easy.. ht eordering and padding are judgement calls..?

In [0]:
conv1.save_weights(model_path + 'conv1.h5')

In [0]:
conv1.load_weights(model_path + 'conv1.h5')

## Pre-trained vectors

You may want to look at wordvectors.ipynb before moving on.

In this section, we replicate the previous CNN, but using pre-trained embeddings.

In [0]:
def get_glove_dataset(dataset):
    """Download the requested glove dataset from files.fast.ai
    and return a location that can be passed to load_vectors.
    """
    # see wordvectors.ipynb for info on how these files were
    # generated from the original glove data.
    md5sums = {'6B.50d': '8e1557d1228decbda7db6dfd81cd9909',
               '6B.100d': 'c92dbbeacde2b0384a43014885a60b2c',
               '6B.200d': 'af271b46c04b0b2e41a84d8cd806178d',
               '6B.300d': '30290210376887dcc6d0a5a6374d8255'}
    glove_path = os.path.abspath('data/glove/results')
    %mkdir -p $glove_path
    return get_file(dataset,
                    'http://files.fast.ai/models/glove/' + dataset + '.tgz',
                    cache_subdir=glove_path,
                    md5_hash=md5sums.get(dataset, None),
                    untar=True)

In [0]:
def load_vectors(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb')),
        pickle.load(open(loc+'_idx.pkl','rb')))

In [0]:
import bcolz
def load_array(fname):
    return bcolz.open(fname)[:]

In [0]:
vecs, words, wordidx = load_vectors(get_glove_dataset('6B.50d')) # observe the names in the cell above
# cehck out the structure of the gove data set

In [164]:
type(vecs)

numpy.ndarray

In [168]:
vecs.view()

array([[ 0.418   ,  0.24968 , -0.41242 , ..., -0.18411 , -0.11514 ,
        -0.78581 ],
       [ 0.013441,  0.23682 , -0.16899 , ..., -0.56657 ,  0.044691,
         0.30392 ],
       [ 0.15164 ,  0.30177 , -0.16763 , ..., -0.35652 ,  0.016413,
         0.10216 ],
       ...,
       [-0.51181 ,  0.058706,  1.0913  , ..., -0.25003 , -1.125   ,
         1.5863  ],
       [-0.75898 , -0.47426 ,  0.4737  , ...,  0.78954 , -0.014116,
         0.6448  ],
       [ 0.072617, -0.51393 ,  0.4728  , ..., -0.18907 , -0.59021 ,
         0.55559 ]], dtype=float32)

In [177]:
isinstance(words, list)
#isinstance(words[0], list)
words[:10]

['the', ',', '.', 'of', 'to', 'and', 'in', 'a', '"', "'s"]

In [182]:
type(wordidx)
#isinstance(wordidx, list)
first2pairs = {k: wordidx[k] for k in wordidx.keys()[:2]}
first2pairs

{'biennials': 130852, 'verplank': 42458}

The glove word ids and imdb word ids use different indexes.  

create a simple function that creates an embedding matrix using the indexes from imdb, and the embeddings from glove (where they exist).

In [0]:
from numpy.random import normal

In [0]:
def create_emb():
    n_fact = vecs.shape[1]# number of factors
    emb = np.zeros((vocab_size, n_fact)) # matrix of zeros:: emb

    for i in range(1,len(emb)):
        word = idx2word[i] # check for word in the idx2 word dict created above
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word): # check that it is a word.. not some spl character junk
            src_idx = wordidx[word] # wordidx is what we got from the glove data set
            emb[i] = vecs[src_idx] # vecs is what we got from the glove datase/ embeddings
        else:
            # If we can't find the word in glove, randomly initialize # cool :)
            emb[i] = normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [0]:
emb = create_emb()

We pass our embedding matrix to the Embedding constructor, and set it to non-trainable.

In [183]:
model = Sequential([
    Embedding(vocab_size, 50, input_length=seq_len, dropout=0.2, 
              weights=[emb], trainable=False),
    Dropout(0.25),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 5, padding="same", activation="relu")`
  """


In [0]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [185]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 9s 371us/step - loss: 0.6557 - acc: 0.5963 - val_loss: 0.5526 - val_acc: 0.7247
Epoch 2/2
25000/25000 [==============================] - 9s 342us/step - loss: 0.5293 - acc: 0.7437 - val_loss: 0.4649 - val_acc: 0.7963


We already have beaten our previous model! But let's fine-tune the embedding weights - especially since the words we couldn't find in glove just have random embeddings.

In [0]:
model.layers[0].trainable=True #the embedding layer

In [0]:
model.optimizer.lr=1e-4 # very tiny LR

In [193]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=1, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 9s 342us/step - loss: 0.4468 - acc: 0.7994 - val_loss: 0.4131 - val_acc: 0.8236


As expected, that's given us a nice little boost. :) From what was 79 .. to 81 on the validation set

In [0]:
model.save_weights(model_path+'glove50.h5')

## Multi-size CNN

This is an implementation of a multi-size CNN as shown in Ben Bowles' [excellent blog post](https://quid.com/feed/how-quid-uses-deep-learning-with-small-data).

In [0]:
from keras.layers import Merge

We use the functional API to create multiple conv layers of different sizes, and then concatenate them.

In [195]:
graph_in = Input ((vocab_size, 50))
convs = [ ] 
for fsz in range (3, 6): 
    x = Convolution1D(64, fsz, border_mode='same', activation="relu")(graph_in)
    x = MaxPooling1D()(x) 
    x = Flatten()(x) 
    convs.append(x)
out = Merge(mode="concat")(convs) 
graph = Model(graph_in, out) 

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same", activation="relu")`
  after removing the cwd from sys.path.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 4, padding="same", activation="relu")`
  after removing the cwd from sys.path.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 5, padding="same", activation="relu")`
  after removing the cwd from sys.path.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  


In [0]:
emb = create_emb()

We then replace the conv/max-pool layer in our original CNN with the concatenated conv layers.  
find out what the graph layer does..?

In [197]:
model = Sequential ([
    Embedding(vocab_size, 50, input_length=seq_len, dropout=0.2, weights=[emb]),
    Dropout (0.2),
    graph,
    Dropout (0.5),
    Dense (100, activation="relu"),
    Dropout (0.7),
    Dense (1, activation='sigmoid')
    ])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  


In [0]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [199]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 21s 848us/step - loss: 0.6521 - acc: 0.5777 - val_loss: 0.3821 - val_acc: 0.8404
Epoch 2/2
25000/25000 [==============================] - 20s 813us/step - loss: 0.3424 - acc: 0.8572 - val_loss: 0.2629 - val_acc: 0.8930


Interestingly, I found that in this case I got best results when I started the embedding layer as being trainable,  
and then set it to non-trainable after a couple of epochs. I have no idea why!

(2 runs of training accuracy goes form 81 to 81.. set the embedding trainable.. accuracy is at 89)

Same here jeremy :P

In [0]:
model.layers[0].trainable=False

In [0]:
model.optimizer.lr=1e-5# even smaller LR

In [202]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 20s 814us/step - loss: 0.2605 - acc: 0.9020 - val_loss: 0.2635 - val_acc: 0.8918
Epoch 2/2
25000/25000 [==============================] - 20s 812us/step - loss: 0.2219 - acc: 0.9178 - val_loss: 0.2606 - val_acc: 0.8958


This more complex architecture has given us another boost in accuracy.
proposal.. run experiments to see if the embedding layer being set to trainable.. boosts the accuracy..  
or it is just a consequence of training the entrie stack again..?

*italicized text*## LSTM

LSTM much mordern.. such wow.. :)

We haven't covered this bit yet!

In [0]:
from keras.regularizers import l2
from keras.layers import  LSTM

In [208]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len, mask_zero=True,
              W_regularizer=l2(1e-6), dropout=0.2),
    LSTM(100, consume_less='gpu'),
    Dense(1, activation='sigmoid')])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary() # one embedding layer.. onle LSTM layer.. for retaining.. memory about the order in which things come in..
#and a dense/ fully connected layer.. 1 output

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(5000, 32, mask_zero=True, embeddings_regularizer=<keras.reg..., input_length=500)`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(100, implementation=2)`
  after removing the cwd from sys.path.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________


In [209]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=5, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 630s 25ms/step - loss: 0.4581 - acc: 0.7695 - val_loss: 0.3280 - val_acc: 0.8642
Epoch 2/5
 9728/25000 [==========>...................] - ETA: 4:54 - loss: 0.2954 - acc: 0.8847

25000/25000 [==============================] - 630s 25ms/step - loss: 0.2849 - acc: 0.8865 - val_loss: 0.3328 - val_acc: 0.8614
Epoch 3/5
22784/25000 [==========================>...] - ETA: 42s - loss: 0.2422 - acc: 0.9067

25000/25000 [==============================] - 629s 25ms/step - loss: 0.2414 - acc: 0.9066 - val_loss: 0.3571 - val_acc: 0.8635
Epoch 4/5
25000/25000 [==============================] - 634s 25ms/step - loss: 0.2230 - acc: 0.9156 - val_loss: 0.3327 - val_acc: 0.8732
Epoch 5/5
 1600/25000 [>.............................] - ETA: 6:59 - loss: 0.1739 - acc: 0.9375

25000/25000 [==============================] - 600s 24ms/step - loss: 0.1876 - acc: 0.9291 - val_loss: 0.3598 - val_acc: 0.8758


LSTM takes time to train.. 1 epoch is about 8 mins to start with..   
5\*8 40 mins.. around :)   
Subsequent layers take lesser time..?

In [210]:
2+2

4